In [1]:
import xml.sax
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/anush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/anush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
filename = '../enwiki-20200801-pages-articles-multistream1.xml-p1p30303'
sergey = '../Sergey_brin.xml'

In [3]:
def cleanupText(arr):
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    x = []
    for stats in arr:
        stats = stats.encode("ascii", errors="ignore").decode()
        stats = re.sub(r'http[^\ ]*\ ',r' ',stats)
        stats = re.sub(r'&nbsp;|&lt;|&gt;|&amp;|&quot;|&apos;', r' ',stats)
        stats = re.sub(r'[^A-Za-z0-9]+',r' ',stats)
        stats = stats.split()
        stats = [i for i in stats if not i in stop_words]
        temp = []
        for words in stats:
            temp.append(ps.stem(words))
        stats = temp
        x.append(stats)
    return tuple(x)

In [4]:
class RegexExp():
    def __init__(self):
        pass
    def findUsefulText(self, info):
        stats = info.split('\n')
        refs = []
        extlinks = []
        cat = []
        for l in stats:
            if re.search(r'ref',l):
                x = re.sub(r'.*title[\ ]*=[\ ]*([^\|]*).*',r'\1',l)
                refs.append(x)
            if re.search(r'\[\[category',l):
                x = re.sub(r'\[\[category:(.*)\]\]',r'\1',l)
                cat.append(x)
            if re.match(r'\*[\ ]*\[',l):
                extlinks.append(l)
        
        return (refs,extlinks,cat)
    
    def findUsefulInfo(self,info):
        stats = info.split('\n')
        i = []
        flag = 0
        
        for l in stats:
            if re.match(r'\{\{infobox', l):
                flag = 1
                x = re.sub('\{\{infobox(.*)\}*\}*',r' ',l)
                i.append(x)
            elif flag == 1:
                if l == '}}':
                    flag = 0
                    continue
                i.append(l)
        stats = cleanupText(' '.join(i))
        return stats
                
        
    
    def textHandling(self,ID,text,title):
        text = text.lower()
        cleaned_text = text.split('==references==')
#         print(data)
        if len(cleaned_text) == 1:
            cleaned_text = text.split('== references== ')
        if len(cleaned_text) == 1:
            refs = []
            extlink = []
            cat = []
        else:
            useful = cleaned_text[1]
            (refs,extlink,cat) = self.findUsefulText(useful)
            (refs,extlink,cat) = cleanupText((' '.join(refs),' '.join(extlink),' '.join(cat)))
        title = title.lower()
        title = cleanupText(title)
        useful = cleaned_text[0]
        info = self.findUsefulInfo(useful)
        stats = re.sub(r'\{\{.*\}\}',r' ',text)
        body = cleanupText(stats)
        
        return (refs,extlink,cat,body,info,title)
        

In [5]:
class XMLHandler(xml.sax.ContentHandler):
    def __init__(self):
        self.CurrentData = ''
        self.title = ''
        self.ID = ''
        self.idFlag = 0
        self.text = ""
        
    def startElement(self,tag,attributes):
        self.CurrentData = tag
            
    def endElement(self,tag):
        if self.CurrentData == 'title':
            pass
#             print(self.title)
        elif self.CurrentData == 'text':
            pass
#             print(self.text)
        
        
        
        if tag == 'page':
            reg = RegexExp()
            refs,extlink,cat,body,info,title = reg.textHandling(self.ID,self.text,self.title)
            self.CurrentData = ''
            self.title = ''
            self.ID = ''
            self.idFlag = 0
            self.text = ''
            
        
        
    def characters(self,content):
        if self.CurrentData == 'title':
            self.title += content
        elif self.CurrentData == 'text':
            self.text += content
        elif self.CurrentData == 'id' and self.idFlag == 0:
            self.ID = content
            self.idFlag = 1

In [6]:
class Parser():
    def __init__(self,filename):
        self.parser = xml.sax.make_parser()
        self.parser.setFeature(xml.sax.handler.feature_namespaces, 0)
        self.handler =  XMLHandler()
        self.parser.setContentHandler(self.handler)
        self.parser.parse(filename)

In [7]:
parser = Parser(filename)

KeyboardInterrupt: 